In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import uniform
from scipy.stats import norm
from scipy.stats import expon
from scipy.optimize import minimize
from mpl_toolkits import mplot3d
%matplotlib inline

In [195]:
def Z(mu, sigma):
    return(norm.cdf(mu/sigma))

def M1(mu, sigma):
    return(1/Z(mu, sigma))

def AR_Norm_Simple(mu, sigma):
    X = -1
    M = M1(mu, sigma)
    while(X == -1):
        Y = mu+sigma*norm.rvs()
        if(Y > 0):
            X = Y
    return(X) 

def params(i, mu, Q, X):
    mu_x = mu[i]-np.sum([Q[i,j]*(X[j]-mu[j]) for j in range(len(mu)) if j != i])/Q[i,i]
    sigma_squared_x = 1/Q[i,i]
    return(mu_x, sigma_squared_x)

def Gibbs_Sampler(n, mu, Sigma, X0):
    d = len(mu)
    Q = np.linalg.inv(Sigma)
    X = [list(X0)]
    for _n in range(n):
        for i in range(d):
            mu_x, s_s_x = params(i, mu, Q, X0)
            X0[i] = AR_Norm_Simple(mu_x, np.sqrt(s_s_x))
        X.append(list(X0))
    return(np.array(X))

def plot_Gibbs(X, mu, Sigma):
    d = len(mu)
    fig, axs = plt.subplots(d, figsize = (10,15))
    x = np.linspace(0, 50, 10000)
    for t in range(d):
        y = norm.pdf(x, loc = mu[t], scale = np.sqrt(Sigma[t,t]))/Z(mu[t], np.sqrt(Sigma[t,t]))
        axs[t].hist(X[5000:,t], density = True, bins = 40, label = "Histogramme des simulations")
        axs[t].plot(x, y, label = "Densité théorique")
    fig.suptitle('Densités théoriques et pratiques', y = 0.9)
    
def autocorr(X, t):
    X_t = np.transpose(X)
    if(t == 0):
        return(np.corrcoef(X_t[:,:],X_t[:,:]))
    return(np.corrcoef(X_t[:,t:],X_t[:,:-t]))

def graph_autocorr(X, t = 50):
    Lags = [_n for _n in range(t+1)]
    d = X.shape[1]
    fig, axs = plt.subplots(d, figsize = (10,15))
    for _d in range(d):
        Auto = [autocorr(X, lag)[_d,_d+3] for lag in Lags]
        axs[_d].plot([-int(t/50),t+int(t/50)],[0,0], color = 'black')
        axs[_d].plot([-int(t/50),t+int(t/50)],[0.2,0.2], color = 'blue', linestyle='dashed')
        axs[_d].plot([-int(t/50),t+int(t/50)],[-0.2,-0.2], color = 'blue', linestyle='dashed')
        for i in range(len(Auto)):
            axs[_d].plot([Lags[i], Lags[i]], [0, Auto[i]], color = 'black')
        axs[_d].axis([-int(t/50),t+int(t/50),min(Auto)-0.1,max(Auto)+0.1])
    fig.suptitle('Graphes des autocorrélations', y = 0.9)
    
def densité(X, mu, Sigma):
    return((np.exp(-(X-mu)*np.linalg.inv(Sigma)*np.transpose(X-mu)+np.exp(-(X-mu)*np.linalg.inv(Sigma_prime)*np.transpose(X-mu))/2)/((2*np.pi)**(len(mu)/2)*np.linalg.det(Sigma)**(0.5))))

In [196]:
n = 10000
mu_prime = np.array([2.,2.,2.])
Sigma_prime = np.array([[2.,1.,1.],
                        [1.,3.,1.],
                        [1.,1.,4.]])
Y0 = expon.rvs(size = 3)
Y = Gibbs_Sampler(n, mu_prime, Sigma_prime, Y0)
#plot_Gibbs(Y, mu_prime, Sigma_prime)
#graph_autocorr(Y)

In [197]:
mu = np.array([1.,2.,3.])
Sigma = np.array([[3.,0.,1.],
                  [0.,4.,2.],
                  [1.,2.,5.]])
X0 = expon.rvs(size = 3)
X = Gibbs_Sampler(n, mu, Sigma, X0)
#plot_Gibbs(X, mu, Sigma)
#graph_autocorr(X)

In [198]:
d = 3

In [222]:
Q_prime = np.linalg.inv(Sigma_prime)
Stock = [np.dot(np.dot((Yi-mu_prime),Q_prime), np.transpose(Yi-mu_prime))/2 for Yi in Y]

$l(x; \mu; \Sigma) = \log(\dfrac{1}
{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})(2\pi)^{N/2} \left| \boldsymbol{\Sigma}\right|^{1/2}}\exp\left[
-\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)\right] \, \mathbb{1}_{x > 0})$

$l(x; \mu; \Sigma) = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(Z(\boldsymbol{\mu}, \boldsymbol{\Sigma}))-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst$

$l(x; \mu; \Sigma) = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\dfrac{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'}))-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst$

or $\dfrac{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})} = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} \dfrac{1}
{(2\pi)^{N/2} \left| \boldsymbol{\Sigma}\right|^{1/2}}\;\exp\left[
-\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)\right] \, \mathbb{1}_{x > 0}\mathrm{d}x = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} 
f(x; \boldsymbol{\mu}, \boldsymbol{\Sigma}) \mathbb{1}_{x > 0}\mathrm{d}x$

$ = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} 
\dfrac{f(x; \boldsymbol{\mu}, \boldsymbol{\Sigma})}{f(x; \boldsymbol{\mu'}, \boldsymbol{\Sigma'})}f(x; \boldsymbol{\mu'}, \boldsymbol{\Sigma'}) \mathbb{1}_{x > 0}\mathrm{d}x = \mathbb{E}\left[\dfrac{f(Y; \boldsymbol{\mu}, \boldsymbol{\Sigma})}{f(Y; \boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\right]$ où $Y \sim \mathcal{N}_{tr}(\mu', \Sigma')$ où $\mu'$ et $\Sigma'$ ne dépendent pas directement de $\mu$ et $\Sigma$ et sont donc constants d'où:

$l(x; \mu; \Sigma) = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\mathbb{E}\left[\dfrac{f(Y; \boldsymbol{\mu}, \boldsymbol{\Sigma})}{f(Y; \boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\right])-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst $

$ = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\dfrac{\left| \boldsymbol{\Sigma'}\right|^{1/2}}{\left| \boldsymbol{\Sigma}\right|^{1/2}}\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right])-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst $

$ = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right])-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|)+\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|)-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma'}\right|) + Cst = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right]) + Cst$

On approximera ensuite $\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right]$ par: $\dfrac{1}{m}\displaystyle \sum_{i = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_i}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_i}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_i}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_i}-\boldsymbol{\mu}\right)}{2}\right]$

Finalement on cherche à minimiser:

$L(\mu, \Sigma) = \displaystyle \sum_{i = 1}^{n} \dfrac{1}{2}\left(\boldsymbol{X_i}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{X_i}-\boldsymbol{\mu}\right) + \log(\displaystyle \sum_{j = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right])$



$\dfrac{\partial L}{\partial \mu}(\mu, \Sigma) = -\displaystyle \sum_{i = 1}^{n} \left(\boldsymbol{X_i}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1} + \dfrac{\displaystyle \sum_{j = 1}^{m} \left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right]}{\displaystyle \sum_{j = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right]}$

In [250]:
Q = np.linalg.inv(Sigma)
expY = np.array([-OP_chiante(Yi, mu, Q) for Yi in Y])
expY_Stock = np.array(expY)+np.array(Stock)
SS = np.array([np.sum([(Q[j,0]*(Yi[j]-mu[j])) for j in range(3)]) for Yi in Y])
S = np.array([np.sum([(Q[j,0]*(Xi[j]-mu[j])) for j in range(3)]) for Xi in X])
np.array(SS)*np.exp(expY)
np.sum(-S+(np.sum(SS*np.exp(expY_Stock)))/np.sum(np.exp(expY_Stock)))

-4.389272633489284

In [241]:
len(S+(np.sum(SS*np.exp(expY)))/np.sum(np.exp(expY)))

10001

In [244]:
(np.sum(SS*np.exp(expY)))/np.sum(np.exp(expY))+S

array([ 1.05792857,  0.30571089,  0.84152575, ...,  0.54575794,
        0.53454416, -0.27900458])

In [227]:
def OP_chiante(Y, mu, Sigma_inv):
    return(np.dot(np.dot(Y-mu, Sigma_inv), np.transpose(Y-mu))/2)

In [231]:
Stock = np.array([OP_chiante(Yi, mu_prime, Q_prime) for Yi in Y])

In [351]:
def l(P):
    mu = P[:d]
    Sigma_raw = P[d:]
    Sigma = np.zeros((d,d))
    for i in range(d):
        Sigma[i, i:d] = Sigma_raw[int(i*d-(i-1)*(i)/2):int(i*d-(i-1)*(i)/2+d-i)]
    Sigma = symmetrize(Sigma)
    Q = np.linalg.inv(Sigma)
    logexpX = np.array([OP_chiante(Xi, mu, Q) for Xi in X])
    expY = np.array([-OP_chiante(Yi, mu, Q) for Yi in Y])
    expY_Stock = np.array(Stock)+np.array(expY)
    return(np.sum(logexpX+np.log(np.sum(np.exp(expY_Stock)))))

In [349]:
Sigma_raw = P0[d:]
Sigma_T = np.zeros((d,d))
for i in range(d):
    a = int(i*d-(i-1)*(i)/2)*(i != 0)
    b = int(i*d-(i-1)*(i)/2+d-i)*(i != 0)+d*(i == 0)
    Sigma_T[i, i:d] = Sigma_raw[a:b]

In [350]:
Sigma_T

array([[3., 0., 1.],
       [0., 4., 2.],
       [0., 0., 5.]])

In [343]:
Sigma

array([[3., 0., 1.],
       [0., 4., 2.],
       [1., 2., 5.]])

In [353]:
Sigma_raw

array([3., 0., 1., 4., 2., 5.])

In [238]:
dl(P0)

-4.3655745685100555

In [220]:
P1 = P0.copy()
P1[0] += 0.00000001

In [221]:
l(P1)

106456.63046195837

In [237]:
def dl(P):
    eps = 1e-10
    P1 = P.copy()
    P1[0] += eps
    return((l(P1)-l(P))/(eps))

In [352]:
mu_f = res.x[:d]
Sigma_f = res.x[d:].reshape(d, d)
print(mu_f, '\n', mu, '\n','\n', Sigma, '\n', Sigma_f)

ValueError: cannot reshape array of size 6 into shape (3,3)

In [360]:
def list_to_sym(d, L):
    s = np.zeros((d,d))
    for i in range(d):
        s[i, i:d] = L[int(i*d-(i-1)*(i)/2):int(i*d-(i-1)*(i)/2+d-i)]
    return(symmetrize(s))

In [361]:
print(res.x[:d], '\n', mu, '\n \n', list_to_sym(d, res.x[d:]), '\n', Sigma)

[0.93101113 1.86616821 2.82399987] 
 [1. 2. 3.] 
 
 [[ 3.19396725 -0.35957819  0.93739825]
 [-0.35957819  5.32136896  2.57746704]
 [ 0.93739825  2.57746704  5.3132971 ]] 
 [[3. 0. 1.]
 [0. 4. 2.]
 [1. 2. 5.]]


In [357]:
d

3

In [354]:
P0 = np.concatenate((mu_prime, np.array(Sigma_raw).flatten()))
res = minimize(l, P0, method = 'BFGS')
print(res)

      fun: 106408.14262463586
 hess_inv: array([[ 2.06832371e-03, -4.01218902e-04,  5.18012360e-04,
        -3.08990064e-03, -5.52632819e-04, -1.26895255e-03,
         1.65451988e-03,  4.73946437e-04, -6.41090985e-04],
       [-4.01218902e-04,  1.64755138e-03,  8.21612882e-04,
         1.61446376e-03, -2.20914068e-03, -5.33788160e-04,
         2.33697070e-04, -7.31923055e-04, -9.88489013e-04],
       [ 5.18012360e-04,  8.21612882e-04,  1.65645001e-03,
        -9.86759481e-05, -1.40267610e-03, -1.60885321e-03,
         6.61282649e-04, -1.27383638e-03, -3.43616863e-03],
       [-3.08990064e-03,  1.61446376e-03, -9.86759481e-05,
         6.95617096e-03, -7.56376845e-04,  1.88318091e-03,
        -3.79675688e-03, -1.91352973e-03,  4.89839850e-05],
       [-5.52632819e-04, -2.20914068e-03, -1.40267610e-03,
        -7.56376845e-04,  6.04790624e-03,  2.74638540e-03,
        -1.17644743e-03,  1.76478426e-03,  2.59992698e-03],
       [-1.26895255e-03, -5.33788160e-04, -1.60885321e-03,
         1

In [268]:
def symmetrize(a):
    return a + a.T - np.diag(a.diagonal())

In [336]:
Sigma_raw = []
for i in range(d):
    Sigma_raw += list(Sigma[i, i:d])